<h2>CS 4780/5780 Final Project: </h2>
<h3>Election Result Prediction for US Counties</h3>

Names and NetIDs for your group members:

<h3>Introduction:</h3>

<p> The final project is about conducting a real-world machine learning project on your own, with everything that is involved. Unlike in the programming projects 1-5, where we gave you all the scaffolding and you just filled in the blanks, you now start from scratch. The programming project provide templates for how to do this, and the most recent video lectures summarize some of the tricks you will need (e.g. feature normalization, feature construction). So, this final project brings realism to how you will use machine learning in the real world.  </p>

The task you will work on is forecasting election results. Economic and sociological factors have been widely used when making predictions on the voting results of US elections. Economic and sociological factors vary a lot among counties in the United States. In addition, as you may observe from the election map of recent elections, neighbor counties show similar patterns in terms of the voting results. In this project you will bring the power of machine learning to make predictions for the county-level election results using Economic and sociological factors and the geographic structure of US counties. </p>
<p>

<h3>Your Task:</h3>
Plase read the project description PDF file carefully and make sure you write your code and answers to all the questions in this Jupyter Notebook. Your answers to the questions are a large portion of your grade for this final project. Please import the packages in this notebook and cite any references you used as mentioned in the project description. You need to print this entire Jupyter Notebook as a PDF file and submit to Gradescope and also submit the ipynb runnable version to Canvas for us to run.

<h3>Due Date:</h3>
The final project dataset and template jupyter notebook will be due on <strong>December 15th</strong> . Note that <strong>no late submissions will be accepted</strong>  and you cannot use any of your unused slip days before.
</p>

![image.png; width="100";](attachment:image.png)

<h2>Part 1: Basics</h2><p>

<h3>1.1 Import:</h3><p>
Please import necessary packages to use. Note that learning and using packages are recommended but not required for this project. Some official tutorial for suggested packacges includes:
    
https://scikit-learn.org/stable/tutorial/basic/tutorial.html
    
https://pytorch.org/tutorials/
    
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html
<p>

In [62]:
import sys
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
# TODO

<h3>1.2 Weighted Accuracy:</h3><p>
Since our dataset labels are heavily biased, you need to use the following function to compute weighted accuracy throughout your training and validation process and we use this for testing on Kaggle.
<p>

In [63]:
def weighted_accuracy(pred, true):
    assert(len(pred) == len(true))
    num_labels = len(true)
    num_pos = sum(true)
    num_neg = num_labels - num_pos
    frac_pos = num_pos/num_labels
    weight_pos = 1/frac_pos
    weight_neg = 1/(1-frac_pos)
    num_pos_correct = 0
    num_neg_correct = 0
    for pred_i, true_i in zip(pred, true):
        num_pos_correct += (pred_i == true_i and true_i == 1)
        num_neg_correct += (pred_i == true_i and true_i == 0)
    weighted_accuracy = ((weight_pos * num_pos_correct) 
                         + (weight_neg * num_neg_correct))/((weight_pos * num_pos) + (weight_neg * num_neg))
    return weighted_accuracy

<h2>Part 2: Baseline Solution</h2><p>
Note that your code should be commented well and in part 2.4 you can refer to your comments. (e.g. # Here is SVM, 
# Here is validation for SVM, etc). Also, we recommend that you do not to use 2012 dataset and the graph dataset to reach the baseline accuracy for 68% in this part, a basic solution with only 2016 dataset and reasonable model selection will be enough, it will be great if you explore thee graph and possibly 2012 dataset in Part 3.

<h3>2.1 Preprocessing and Feature Extraction:</h3><p>
Given the training dataset and graph information, you need to correctly preprocess the dataset (e.g. feature normalization). For baseline solution in this part, you might not need to introduce extra features to reach the baseline test accuracy.
<p>

In [64]:
# You may change this but we suggest loading data with the following code and you may need to change
# datatypes and do necessary data transformation after loading the raw data to the dataframe.
dataset_train_path = "train_2016.csv"
dataset_test_path = "test_2016_no_label.csv"
df = pd.read_csv(dataset_train_path, sep=',',header=None,encoding='unicode_escape')
df2 = pd.read_csv(dataset_test_path, sep=',',header=None,encoding='unicode_escape')

In [65]:
#Preprocessing Train Dataset
#create a new column for labels
df[10] = 0
df[10][0] = 'Predictions'

#fill in the new column: Label == 1 if DEM > GOP, Label == 0 otherwise
for i in range(1, len(df)):
    if (int(df.loc[i, 2])) > (int(df.loc[i, 3])): df.loc[i, 10] = 1

#Change Columns names from # -> String
for i in range(len(df.loc[0])):df = df.rename(columns= {i:df.loc[0][i]})

#Remove unnecessary 0th row
df = df.drop([0])

#remove thousand comma
df['MedianIncome']=df['MedianIncome'].str.replace(',','')

#type conversions
df = df.astype({'MedianIncome': int, 'MigraRate': float,'BirthRate': float,'DeathRate': float,'BachelorRate': float,'UnemploymentRate': float})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [66]:
df.head()

,FIPS,County,DEM,GOP,MedianIncome,MigraRate,BirthRate,DeathRate,BachelorRate,UnemploymentRate,Predictions
1,18019,"Clark County, IN",18791,30012,51837,4.9,12.8,11.0,20.9,4.2,0
2,6035,"Lassen County, CA",2026,6533,49793,-18.4,9.2,6.3,12.0,6.9,0
3,40081,"Lincoln County, OK",2423,10838,44914,-1.3,11.4,11.7,15.1,5.3,0
4,31153,"Sarpy County, NE",27704,44649,74374,9.2,14.2,5.0,40.1,2.9,0
5,28055,"Issaquena County, MS",395,298,26957,-12.8,9.8,5.3,6.7,14.0,1


In [67]:
df2.head()

,0,1,2,3,4,5,6,7
0,FIPS,County,MedianIncome,MigraRate,BirthRate,DeathRate,BachelorRate,UnemploymentRate
1,17059,"Gallatin County, IL","39,634",-10.6,10.8,12.5,9.9,7.6
2,6103,"Tehama County, CA","40,585",1.8,12.8,10.4,15.5,7.1
3,42047,"Elk County, PA","49,274",-9.3,9.7,13,18.6,5.5
4,47147,"Robertson County, TN","58,487",7.4,12.7,9.7,18.6,4.1


In [68]:
#Preprocessing for Test DataSet

#Change Column names from # -> String
for i in range(len(df2.loc[0])):df2 = df2.rename(columns= {i:df2.loc[0][i]})

#Remove unnecessary 0th row
df2 = df2.drop([0])

#remove thousand comma
df2['MedianIncome']=df2['MedianIncome'].str.replace(',','')

#type conversions
df2 = df2.astype({'MedianIncome': int, 'MigraRate': float,'BirthRate': float,'DeathRate': float,'BachelorRate': float,'UnemploymentRate': float})

In [69]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        if (feature_name == 'MedianIncome' or feature_name == 'MigraRate' or feature_name == 'BirthRate' or feature_name == 'DeathRate' or feature_name == 'BachelorRate' or feature_name == 'UnemploymentRate'):
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [70]:
df_train = normalize(df)
df_test = normalize(df2)
print(df_train)

       FIPS                  County  ... UnemploymentRate Predictions
1     18019        Clark County, IN  ...         0.111607           0
2      6035       Lassen County, CA  ...         0.232143           0
3     40081      Lincoln County, OK  ...         0.160714           0
4     31153        Sarpy County, NE  ...         0.053571           0
5     28055    Issaquena County, MS  ...         0.549107           1
...     ...                     ...  ...              ...         ...
1551  36009  Cattaraugus County, NY  ...         0.191964           0
1552  55031      Douglas County, WI  ...         0.156250           1
1553  27065      Kanabec County, MN  ...         0.214286           0
1554  17139     Moultrie County, IL  ...         0.129464           0
1555  20185     Stafford County, KS  ...         0.089286           0

[1555 rows x 11 columns]


<h3>2.2 Use At Least Two Training Algorithms from class:</h3><p>
You need to use at least two training algorithms from class. You can use your code from previous projects or any packages you imported in part 1.1.

In [71]:
#More packages for Algorithms:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [72]:
# Make sure you comment your code clearly and you may refer to these comments in the part 2.4
# TODO
# perceptron object with the parameters: 60 iterations (epochs) over the data, and a learning rate of 0.15
ppn = Perceptron(max_iter=60, eta0=0.15, random_state=0)


knn = KNeighborsClassifier(n_neighbors=3)

<h3>2.3 Training, Validation and Model Selection:</h3><p>
You need to split your data to a training set and validation set or performing a cross-validation for model selection.

In [73]:
# Splitting training data into train and test
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns = ['DEM', 'GOP','County', 'Predictions', 'FIPS']), df_train['Predictions'], test_size=0.3)
y_train=y_train.astype('int')
y_test = y_test.to_numpy()

ppn.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [74]:
#Predictions made from Perceptron
y_pred_ppn = ppn.predict(X_test)
#Predictions made from KNN
y_pred_knn = knn.predict(X_test)

In [80]:
print(weighted_accuracy(y_pred_ppn, y_test))

0.7610386259767064


In [76]:
print(weighted_accuracy(y_pred_knn, y_test))

0.7225969335102462


<h3>2.4 Explanation in Words:</h3><p>
    You need to answer the following questions in the markdown cell after this cell:

2.4.1 How did you preprocess the dataset and features?

**One of the things we did to preprocess our dataset and features was adding a column "Predictions" which was a binary label of 0 if a county was Republican and 1 if a county was Democrat. This allowed us to more easily train our model, according to the binary classification of each county in the training set. Additionally, we removed all commas from numbers in the dataset and casted all rows in the dataset to make sure each feature was of the correct type. We also removed the first row of the datasets which had each column name and moved this up so that the dataset's column names were the actual column name strings as opposed to column indices. This made accessing each column much simpler.**

2.4.2 Which two learning methods from class did you choose and why did you made the choices?

**We chose to use KNN and the Perceptron learning methods, both of which are supervised learning approaches. We went with KNN because of its simplicity and we believed that it was likely that counties that were closer to each other (by distance) when considering characteristic dimensions such as unemployment rate and median income, would probably vote similarly. KNN takes this into account by selecting the label of the closest neighbor counties. We also chose to try the Perceptron learning method, primarily because of its ease of implementation. We knew that Perceptrons are only successful when the data is linearly separable, so we felt that trying this approach would tell us whether this was the case or not.**

2.4.3 How did you do the model selection?

2.4.4 Does the test performance reach a given baseline 68% performance? (Please include a screenshot of Kaggle Submission)

<h2>Part 3: Creative Solution</h2><p>

<h3>3.1 Open-ended Code:</h3><p>
You may follow the steps in part 2 again but making innovative changes like creating new features, using new training algorithms, etc. Make sure you explain everything clearly in part 3.2. Note that reaching the 75% creative baseline is only a small portion of this part. Any creative ideas will receive most points as long as they are reasonable and clearly explained.

In [77]:
# Make sure you comment your code clearly and you may refer to these comments in the part 3.2

from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler # maybe add this to preprocessing step
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# C_range = np.logspace(-2, 10, 5)
# gamma_range = np.logspace(-9, 3, 5)
# param_grid = dict(gamma=gamma_range, C=C_range)
# grid = GridSearchCV(SVC(), param_grid=param_grid)
# grid.fit(X_train_scaled, y_train)

params = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
rbf_svm = SVC(gamma='auto')
grid = GridSearchCV(rbf_svm, params)

grid.fit(X_train_scaled, y_train)



GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [78]:
# Predictions made by RBF SVM approach
y_pred_svm = grid.predict(X_test_scaled)

In [79]:
print(weighted_accuracy(y_pred_svm, y_test))

0.6847449506118237


<h3>3.2 Explanation in Words:</h3><p>

You need to answer the following questions in a markdown cell after this cell:

3.2.1 How much did you manage to improve performance on the test set compared to part 2? Did you reach the 75% accuracy for the test in Kaggle? (Please include a screenshot of Kaggle Submission)

3.2.2 Please explain in detail how you achieved this and what you did specifically and why you tried this.

<h2>Part 4: Kaggle Submission</h2><p>
You need to generate a prediction CSV using the following cell from your trained model and submit the direct output of your code to Kaggle. The CSV shall contain TWO column named exactly "FIPS" and "Result" and 1555 total rows excluding the column names, "FIPS" column shall contain FIPS of counties with same order as in the test_2016_no_label.csv while "Result" column shall contain the 0 or 1 prdicaitons for corresponding columns. A sample predication file can be downloaded from Kaggle.

In [ ]:
# TODO

# You may use pandas to generate a dataframe with FIPS and your predictions first 
# and then use to_csv to generate a CSV file.

<h2>Part 5: Resources and Literature Used</h2><p>